# Option Basics and the Black-Scholes-Merton Model

**Table of Contents**
 * [Simple derivatives](#section_derivatives)
    * [Forwards](#subsection_forwards)
    * [Call Options](#subsection_calls)
    * [Put Options](#subsection_puts)


## Simple derivatives

#### Recap Discounting

Why do we use discounting?

There are riskless investments on the market, e.g. investing in Treasury bonds. <br>
If we have \\$100 dollars, we can do the followings:
- put it in the freezer, so at any time, we can defrost it and have \\$100.
- invest in Treasury Bonds, which gives us $r$ coupon per annum, so we'll have \\$100 + \\$$\frac{r}{100}$ one year from now

So if we lend money, we should always think about that investing the same amount would generate us income, and if the investment is riskless, this income is guaranteed. <br>
That is why money has a *time value*.

Let assume $r$ is the risk-free interest rate, at which money is borrowed or lent with no credit risk, so the money is certain to be paid.

We have $N$ amount of money to invest for $t$ years, then our payoff depends on the compound frequency:

<table>
  <tr>
    <th>Investing the money every year</th>
    <th></th>
    <th>Investing the money $m$ times a year</th>
    <th></th>
    <th>Investing the money continuously</th>
  </tr>
  <tr>
    <td>
        \begin{equation}
            N \cdot (1 + r)^t
        \end{equation}
    </td>
    <td></td>
    <td> 
        \begin{equation}
            N \cdot \left( 1 + \frac{r}{m}\right) ^{m \cdot t}
        \end{equation}
    </td>
    <td></td>
    <td> 
        \begin{equation}
            N \cdot e^{r\cdot t}
        \end{equation}
    </td>
  </tr>
</table>

$N$ is called the **present value** of our investment, while $N \cdot e^{r\cdot t}$ is the **future value**.

I.e. a price of a bond which pays $n$ cachflows during its life has a present value of
\begin{equation}
            PV = \sum_{i=1}^n Cashflow_i \cdot e^{-r_i \cdot t_i}
\end{equation}

### Forwards <a class="anchor" id="subsection_forwards"></a>

**Spot contract**
: an agreement to buy or sell an asset, e.g. a stock almost immediately. Its price is called **spot price**, and we refer to it as $S_0$

**Forward contract**
: an agreement to buy or sell an asset at a certain future time for a certain price. This date when the sale is made is called **maturity** (or expiry) of the contract, and it is denoted by $T$. <br>
$S_T$ is the spot price of the underlying at maturity $T$.

One of the parties assumes a long position of the forward contract and agrees to buy the underlying asset on that future date for the predefined price, and the other party assumes a short position and agrees to sell the asset on the same date for the same price.

#### What is the payoff of this forward contract?

The **payoff** from a long position on one unit of an asset is
\begin{equation}
S_T - K
\end{equation}
where $K$ is the delivery price. It means that the buyer pays $K$ for the asset, and then receives the asset itself, which worth $S_T$ at that time. This amount can be either positive or negative, depending on the price of the asset at $T$.

Similarly, the payoff from a short position on one unit of an asset is
\begin{equation}
K - S_T
\end{equation}

<br>

<table>
  <tr>
    <th>Payoff from a long position</th>
    <th>Payoff from a short position</th>
  </tr>
  <tr>
    <td><img src="images/long_forward_payoff.PNG" width="400"/></td>
    <td><img src="images/short_forward_payoff.PNG" width="400"/></td>
  </tr>
</table>

#### What is the value of this forward contract at time $t$?
At time $0$,
\begin{equation}
f(0, S_0) = S_0 - e^{-rT} \cdot K
\end{equation}

Imagine that we sell this forward contract which leads to the income of $S_0 - e^{-rT}\cdot K$, but then at time $T$, we must sell the underlying stock. Thus we need to borrow $ e^{-rT}\cdot K$ from the market, to have enough money to buy one unit.
At expiry, we'll have a stock which is worth $S_T$ and a debt worth of $K$, so we replicated the payoff of the forward contract. It means that during the lifetime of the contract, the **value of the forward** contract must be the same as the value of this portfolio, which means at any time $t$

\begin{equation}
f(t, S_t) = S_t - e^{-r(T-t)} \cdot K
\end{equation}



#### What if the price of this portfolio does not match the forward value?

E.g. if $f(0, S_0) > S_0 - e^{-rT} \cdot K$, then we can sell the forward contract for $f(0, S_0)$ and buy the same porfolio for $S_0 - e^{-rT} \cdot K$, which leads to an instant income.

This is called an **arbitrage** opportunity on the market.

**Arbitrage**
: a simultaneous purchase and sale of the same (or very similar) product on different markets in order to gain from the price diferences, which is considered as a riskless profit.

#### What is the delivery price of this forward contract?

Entering into a forward contract does not need any money exchange, no cash is flowing between the parties till the expiry of the contract. Thus it does not worth anything at time 0.

It means that $K$ is determined to satisfy $S_0 - e^{-rT} \cdot K = 0$, thus:
\begin{equation}
K = S_0 \cdot e^{rT} =: F_0^T
\end{equation}

This $F_0^T$ is called the **forward price** at time $0$.

If $K$ does not satifies the above equality, it leads to arbitrage oppurtunities.

#### What is the difference between the value of forward and the price of forward?

The forward price is something fixed at the initalization of the contract, while the forward value changes during the life of the contact: it is zero initially at time $0$, but changes to either positive or negative as the market evolves to time $T$. 

### Call Options <a class="anchor" id="subsection_calls"></a>

**Call option**
: gives the option holder the right to *buy* the underlying asset by a certain date for a certain price. This predefined price for sale is called **strike price**. The option seller on the other hand must *sell* the underlying if the option is exercised.

The date when the option is exercised can be very different depending on the style of the trade. The most simple construction is called **European call option**, when the option can be exercised only on a predefined date, which is called the **maturity** (or exercise date) of the option.

IMPORTANT: the holder does not have to exersice the option, only if he/she wants.

#### What is the benefit of this construction?

Let's assume that, we received a call option as a gift which says if we want, we can buy a stock of Abracadabra Inc. for \\$100 one year from now. <br>
Let's think through the following scenarios:

- Scenario A: the price of the stock rises to $120 in a year

- Scenario B: the price of the stock falls to $90 in a year

In Scenario A, if we exercise the option and buy a stock for \\$100, we can earn \\$20 by selling the stock immediately. In Scenario B, either we buy the stock for \\$100 from the option seller or buy it on the market for \\$90. In this case, if we exercise the option, we'll technically lose \\$10. <br>
Since the call option is not a forward contract, we can decide to not buy the stock in Scenario B to lose money, which is a clear benefit.

Let's think from the sellers point of view. In Scenario B, nothing will happen, but in Scenario A, they would lose money because they have to sell the stock for \\$100 instead of \\$120 on the market. This means that the option seller takes the risk to lose money, for which they can claim a price when they sell the option. Let's denote this **call option price** with $c$.


#### What is the payoff of this call option contract?

The payoff of the call option is 
\begin{equation}
max\lbrace S_T - K, 0\rbrace
\end{equation}

<br>

<table>
  <tr>
    <th>Payoff of a call option</th>
  </tr>
  <tr>
    <td><img src="images/call_option_payoff.PNG" width="400"/></td>
  </tr>
</table>

### Put Options <a class="anchor" id="subsection_puts"></a>

**Put option**
: gives the option holder the right to *sell* the underlying asset by a certain date for a certain price. Similarly to the call option, this predefined price is called **strike price**. The option seller on the other hand must *buy* the underlying if the option is exercised.

Analogously, a **European put option** can be only exercised on the maturity date of the option.

#### What is the benefit of this construction?

The option seller takes risk with a put option as well, so they will claim a price for it. Let's denote this **put option price** with $p$.

Let's assume that we just bought an Abracadabra Inc. stock from the market which worth \\$100 now, and we bought a put option which says we'll have the chance to sell the stock for \\$100 one year from now. <br>
Let's think through the same scenarios:

- Scenario A: the price of the stock rises to $120 in a year

- Scenario B: the price of the stock falls to $90 in a year

In Scenario A, we should not exercise our option, since , we would sell our stock for \\$100 instead of \\$120 market price. In contract, in Scenario B,
we can sell our stock to the option seller for higher price than the market price. We can even buy the predefined amount of stocks on the market at time $T$ for \\$90 /unit, exercise the option and sell them to the option seller for \\$100.

Put options also act as something like an insurance, to floor your loss if the stock price falls significatly.


#### What is the payoff of this put option contract?
The payoff of the put option is 
\begin{equation}
min\lbrace K - S_T, 0\rbrace
\end{equation}

<br>

<table>
  <tr>
    <th>Payoff of a put option</th>
  </tr>
  <tr>
    <td><img src="images/put_option_payoff.PNG" width="400"/></td>
  </tr>
</table>

## Relationship of Forward, Call and Put prices

Consider the following two portfolios:
- Portfolio X: one European call option + a zero-coupon bond that provides a payoff of $K$ at time $T$

- Portfolio Y: one European put option + one share of the stock

#### What are the payoffs of these portfolios?

<table>
    <tr>
        <td>
            <table>
                <tr>
                    <th></th>
                    <th></th>
                    <th>$S _T > K$</th>
                    <th>$S _T < K$</th>
                </tr>
                <tr>
                    <td rowspan="3">Portfolio X</td>
                    <td>Call option</td>
                    <td>$S_T - K$</td>
                    <td>$0$</td>
                </tr>
                <tr>
                    <td>Zero-coupon bond</td>
                    <td>$K$</td>
                    <td>$K$</td>
                </tr>
                <tr>
                    <td><i>Total</i></td>
                    <td>$S_T$</td>
                    <td>$K$</td>
                </tr>
            </table>
        </td>
        <td></td>    
        <td>
            <table>
                <tr>
                    <th></th>
                    <th></th>
                    <th>$S _T > K$</th>
                    <th>$S _T < K$</th>
                </tr>
                <tr>
                    <td rowspan="3">Portfolio Y</td>
                    <td>Put option</td>
                    <td>$0$</td>
                    <td>$K - S_T$</td>
                </tr>
                <tr>
                    <td>Stock</td>
                    <td>$S_T$</td>
                    <td>$S_T$</td>
                </tr>
                <tr>
                    <td><i>Total</i></td>
                    <td>$S_T$</td>
                    <td>$K$</td>
                </tr>
            </table>
        </td>
    </tr>
</table>

So the payoff of both portfolios when the option expire at time $T$ is
\begin{equation}
max\lbrace S_T, K\rbrace
\end{equation}

#### Put-Call Parity
If we want to avoid arbitrage possibilities, the portfolios must have identical values today. <br>
The components of Portfolio X are worth $c$ and $K\cdot e^{-rT}$ today, and the components of Portfolio Y are worth $p$ and $S_0$ today.

Hence the following applies:
\begin{equation}
c + K\cdot e^{-rT} = p + S_0
\end{equation}

For any $ 0 \leq t \leq T$:
\begin{equation}
c_t + K\cdot e^{-r\cdot(T-t)} = p_t + S_t
\end{equation}

which equality is called **Put-Call Parity**.

It shows that we can deduce the price of the put from the price of the call with the same strike price and maturity date, and vica versa.

With a small rearrangement, we'll get the following equality:

\begin{equation}
c_t - p_t = S_t - K\cdot e^{-r\cdot(T-t)}
\end{equation}

which shows that a long call plus a short put option payoff is the same as a long forward payoff.

<table>
  <tr>
    <th>Put-Call Parity</th>
  </tr>
  <tr>
    <td><img src="images/put_call_parity.PNG" width="600"/></td>
  </tr>
</table>